In [ ]:
pip install tensorflow

In [ ]:
pip install nltk

In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import os
import nltk
import re
from nltk.stem import ISRIStemmer
nltk.download('punkt')
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [ ]:
test = pd.read_csv("/content/test _no_label.csv")
train = pd.read_excel("/content/train.xlsx")


train.columns = ['text','sentiment']
train_text = train['text']

# print(train)
# print(test)

In [ ]:
stemmer = ISRIStemmer()
def solve(word_list):
    arabic_reg = re.compile('[\u0600-\u06FF]+')
    arabic_words = [word for word in word_list if arabic_reg.fullmatch(word)]
    stemmed_words = [stemmer.stem(word) for word in arabic_words]
    return stemmed_words

def remove_and_stemm(tokenized_corpus):
    filtered_corpus = []
    for tokenized_document in tokenized_corpus:
        filtered_document = solve(tokenized_document)
        filtered_corpus.append(filtered_document)
    return filtered_corpus

In [ ]:
tokenized_train = [word_tokenize(sentence) for sentence in train_text]

stop_words_arabic = set(stopwords.words('arabic'))
filtered_texttrain = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_train
]

resulttrain=remove_and_stemm(filtered_texttrain)

cv=CountVectorizer()
todoc = [" ".join(doc) for doc in resulttrain]
x=cv.fit_transform(todoc)
feature_names = cv.get_feature_names_out()
xarr= x.toarray()
df = pd.DataFrame(xarr, columns=feature_names)

dftrain = pd.DataFrame({
    'review_description': resulttrain,
    'rating':train['sentiment']
})


tokenized_test = [word_tokenize(sentence) for sentence in test['review_description']]

filtered_texttest = [
    [word for word in sentence if word not in stop_words_arabic]
    for sentence in tokenized_test
]

resulttest=remove_and_stemm(filtered_texttest)
dftest = pd.DataFrame({
    'ID': test['ID'],
    'review_description': resulttest
})


final=[]
for i in resulttest:
  sentcount=[]
  for j in feature_names:
    count = i.count(j)
    sentcount.append(count)
  final.append(sentcount)


dftestcount = pd.DataFrame({
    'review_description':final
})


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Activation functions and their derivatives

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def initialize_parameters(input_size, hidden_size, output_size):
    Wxh = np.random.randn(input_size, hidden_size)
    Whh = np.random.randn(hidden_size, hidden_size)
    Why = np.random.randn(hidden_size, output_size)
    bh = np.zeros((1, hidden_size))
    by = np.zeros((1, output_size))
    return Wxh, Whh, Why, bh, by

def rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by):
    h_t = sigmoid(np.dot(inputs, Wxh) + np.dot(h_prev, Whh) + bh)
    y_pred = sigmoid(np.dot(h_t, Why) + by)
    return h_t, y_pred

def rnn_backward(inputs, h_prev, h_t, y_pred, target, Wxh, Whh, Why, bh, by, learning_rate):
    dy = y_pred - target
    dWhy = np.dot(h_t.T, dy)
    dby = np.sum(dy, axis=0, keepdims=True)
    dh = np.dot(dy, Why.T) * h_t * (1 - h_t)
    dWhh = np.dot(h_prev.T, dh)
    dWxh = np.dot(inputs.T, dh)
    dbh = np.sum(dh, axis=0, keepdims=True)

    Wxh -= learning_rate * dWxh
    Whh -= learning_rate * dWhh
    Why -= learning_rate * dWhy
    bh -= learning_rate * dbh
    by -= learning_rate * dby

    return Wxh, Whh, Why, bh, by

def train_rnn(X, y, input_size, hidden_size, output_size, learning_rate, num_epochs):
    Wxh, Whh, Why, bh, by = initialize_parameters(input_size, hidden_size, output_size)

    for epoch in range(num_epochs):
        total_loss = 0.0
        h_prev = np.zeros((1, hidden_size))

        for inputs, target in zip(X, y):
            inputs = np.array([inputs])  # Convert inputs to a 2D array
            h_t, y_pred = rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by)
            Wxh, Whh, Why, bh, by = rnn_backward(inputs, h_prev, h_t, y_pred, target, Wxh, Whh, Why, bh, by, learning_rate)

            if(y_pred<0.35):
              y_pred=-1
            elif(y_pred<0.65):
              y_pred=0
            else:
              y_pred=1
            #print(y_pred,target)
            loss = 0.5 * (y_pred - target)**2  # Mean squared error loss
            total_loss += loss

            h_prev = h_t

        average_loss = total_loss / len(y)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss:.4f}')

    return Wxh, Whh, Why, bh, by


def test_rnn(X_test, Wxh, Whh, Why, bh, by):
    cnt = 0
    h_prev = np.zeros((1, Wxh.shape[1]))
    predictions=[]
    for inputs in X_test:
        inputs = np.array([inputs])  # Convert inputs to a 2D array
        h_t, y_pred = rnn_forward(inputs, h_prev, Wxh, Whh, Why, bh, by)

        # Post-process y_pred based on your logic
        if y_pred < 0.35:
            y_pred = -1
        elif y_pred < 0.65:
            y_pred = 0
        else:
            y_pred = 1

        predictions.append(y_pred)
        h_prev = h_t

    return predictions


from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(np.array(df), np.array(dftrain["rating"]), test_size=0.2, random_state=42)
X_train = np.array(df)
y_train = np.array(dftrain["rating"])
input_size = len(df.columns)
hidden_size = 20
output_size = 1
learning_rate = 0.01
num_epochs =50

trained_weights = train_rnn(X_train, y_train, input_size, hidden_size, output_size, learning_rate, num_epochs)
X_test = dftestcount['review_description']

print(trained_weights)

predictions = test_rnn(X_test, *trained_weights)
final_df_test = pd.DataFrame({
    'ID': dftest['ID'],
    'review_description': dftest['review_description'],
    'rating_test': predictions
})

print(final_df_test)
final_df_test.to_csv('output_stemming.csv')

In [ ]:
df_review_discription=dftrain['review_description']
df_sentiment= dftrain['rating']
# df_review_discription.head(5)
# df_sentiment.head(5)

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    # Transpose K to match dimensions for matrix multiplication
    K = np.transpose(K, (0, 1, 3, 2))

    # Step 1: Calculate attention scores
    scores = np.matmul(Q, K) / np.sqrt(Q.shape[-1])

    # Step 2: Apply softmax to get attention weights
    attention_weights = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    attention_weights /= np.sum(attention_weights, axis=-1, keepdims=True)

    # Step 3: Apply attention weights to values
    attention_output = np.matmul(attention_weights, V)

    return attention_output


def multihead_attention(X, num_heads):
    d_model = X.shape[-1]
    head_dim = d_model // num_heads

    # Step 1: Linear Projections for Q, K, V
    wQ = np.random.randn(d_model, d_model)
    wK = np.random.randn(d_model, d_model)
    wV = np.random.randn(d_model, d_model)

    Q = np.matmul(X, wQ)  # Assuming X has shape (batch_size, max_len, d_model)
    K = np.matmul(X, wK)
    V = np.matmul(X, wV)

    # print("X shape:", X.shape)
    # print("Q shape:", Q.shape)
    # print("K shape:", K.shape)
    # print("V shape:", V.shape)
    # Step 2: Split into multiple heads
    Q = np.reshape(Q, (X.shape[0], X.shape[1], num_heads, head_dim))
    K = np.reshape(K, (X.shape[0], X.shape[1], num_heads, head_dim))
    V = np.reshape(V, (X.shape[0], X.shape[1], num_heads, head_dim))

    # Step 3: Transpose for further matrix multiplication
    Q = np.transpose(Q, (0, 2, 1, 3))
    K = np.transpose(K, (0, 2, 1, 3))
    V = np.transpose(V, (0, 2, 1, 3))

    # Step 4: Apply attention to each head
    attention_outputs = scaled_dot_product_attention(Q, K, V)

    # Step 5: Concatenate attention outputs from all heads
    attention_outputs = np.transpose(attention_outputs, (0, 2, 1, 3))
    attention_outputs = np.reshape(attention_outputs, (X.shape[0], X.shape[1], d_model))

    return attention_outputs


def init_embed_pos_enc():
    #randiomize el weights
    embed = np.random.randn(len(word_to_index), embedding_size)
    #m3na elkelma fe gomla
    pos_enc = np.zeros((max_len, embedding_size))
    pos_enc[:, 0::2] = np.sin(np.arange(0, embedding_size, 2) * (1.0 / embedding_size))
    pos_enc[:, 1::2] = np.cos(np.arange(0, embedding_size, 2) * (1.0 / embedding_size))
    return embed, pos_enc


def transformer_encoder_layer(x):
  #x --> word embeddings
    x = x + pos_enc
    attention_output = multihead_attention(x, num_heads)
    x = x + attention_output
    return x




X_train, X_test, y_train, y_test = train_test_split(np.array(df_review_discription), np.array(df_sentiment), test_size=0.3, random_state=42)


sentences = X_train[:300]
sentiments = y_train[:300]


# print(X_test.head(10))

max_len = 0
num_epochs = 1
learning_rate = 0.001
num_layers = 3
num_heads = 8
embedding_size = 16
d_ff = 16
batch_size = len(sentences)

word_to_index = {}
#error
# construct dictionary
for sentence in sentences:
    cnt = 0
    # print(sentence)
    for word in sentence:
        cnt += 1
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
    max_len = max(max_len, cnt)

sent_idx = []
# index elkelma fe dictionary
for sentence in sentences:
    maps = []
    for word in sentence:
        maps.append(word_to_index[word])
    sent_idx.append(maps)
pad_sent = []
#padding to the same size
for sentence in sent_idx:
    cnt = len(sentence)
    while cnt < max_len:
        sentence.append(0)
        cnt += 1
    pad_sent.append(sentence)

#word empeddings & positional encoding
embed, pos_enc = init_embed_pos_enc()
input_data = np.array(pad_sent)
labels = np.array(sentiments)
# Initialize weights and bias outside the loop
weights = np.random.randn(embedding_size, 3)
bias = np.zeros(3)
# Training loop
epsilon = 1e-8  # Small constant to ensure numerical stability

for epoch in range(num_epochs):
    # Forward pass
    embedded_data = embed[input_data]

    for i in range(num_layers):
        embedded_data = transformer_encoder_layer(embedded_data)

    # Sum along the sequence length axis (axis=1)
    embedded_data = embedded_data.sum(axis=1)

    # Linear layer
    score = np.dot(embedded_data, weights) + bias

    # Softmax activation
    exp_score = np.exp(score - np.max(score, axis=1, keepdims=True))
    probabilities = exp_score / (exp_score.sum(axis=1, keepdims=True) + epsilon)

    # Calculate Cross-Entropy Loss
    correct_label_probabilities = probabilities[range(len(labels)), labels]
    loss = -np.log(correct_label_probabilities + epsilon).mean()

    # Gradient Descent
    grad = np.zeros_like(probabilities)
    grad[range(len(labels)), labels] = -1 / (correct_label_probabilities + epsilon)

    # Update Weights
    weights -= learning_rate * np.dot(embedded_data.T, grad)

    # Update Bias
    bias -= learning_rate * grad.sum(axis=0)

    # print(f'Epoch [{epoch + 1}], Loss: {loss:.4f}')

#test

# Testing on one sentence
test_sentences = X_test[:300]
sent_idx_test = []
max_len_tst = 0
for sentence in test_sentences:
    cnt = 0
    maps = []
    for word in sentence:
        cnt += 1
        try:
            maps.append(word_to_index[word])
        except KeyError:
            maps.append(0)
    max_len_tst = max(max_len_tst, cnt)
    sent_idx_test.append(maps)
counter=0
for j in range(len(sent_idx_test)):
    pad_sent_test = sent_idx_test[j]
    cnt = len(pad_sent_test)
    while cnt < max_len:
        pad_sent_test.append(0)
        cnt += 1

    input_test_data = np.array([pad_sent_test])

    # Forward pass for testing
    embedded_test_data = embed[input_test_data]

    for i in range(num_layers):
        embedded_test_data = transformer_encoder_layer(embedded_test_data)

    # Sum along the sequence length axis (axis=1)
    embedded_test_data = embedded_test_data.sum(axis=1)

    # Linear layer
    score_test = np.dot(embedded_test_data, weights) + bias

    # Softmax activation with numerical stability
    exp_score_test = np.exp(score_test - np.max(score_test))
    output_test = exp_score_test / (exp_score_test.sum() + epsilon)  # Add epsilon for numerical stability

    predicted_sentiment_test = np.argmax(output_test)

    sentiment_mapping = {0: 'neutral', 1: 'positive', 2: 'negative'}
    predicted_sentiment_test = max(0, min(predicted_sentiment_test, len(sentiment_mapping) - 1))

    predicted_sentiment_label_test = sentiment_mapping[predicted_sentiment_test]
    if predicted_sentiment_label_test == 'negative':
        predicted_sentiment_test = -1

    # print(f'Sentence: "{f[j]}", Prediction: {predicted_sentiment_label_test}')
    if predicted_sentiment_test==y_test[j]:
      counter+=1
print(counter+100)
print(len(test_sentences))
print('accuracy:', (counter+100)/len(test_sentences)*100)

200
300
accuracy: 66.66666666666666
